# Scaled System Example

## Prob  3.5.3

We have matrix $A$ and $A^{-1}$. The inverse $A^{-1}$ is the mathematical solution.

$$
A = \begin{bmatrix}
2 & -1 & 1 \\
-1 & 10^{-10} & 10^{-10} \\
1 & 10^{-10} & 10^{-10}
\end{bmatrix}
$$

$$
A^{-1} = \frac{1}{2} \begin{bmatrix}
0 & -1 & 1 \\
-1 & 0.5 \times 10^{10} - 1 & 0.5 \times 10^{10} + 1 \\
1 & 0.5 \times 10^{10} + 1 & 0.5 \times 10^{10} - 1
\end{bmatrix}
$$

## Find Condition Number $k_\infty (A)$

The condition number k(A) is defined as:

$$
k(A) := \|A\| \|A^{-1}\|
$$

Note: The computed $A^{-1}$ may not be accurate due to numerical instability, especially for ill-conditioned matrices.

In [1]:
import numpy as np
from numpy.linalg import inv, pinv, LinAlgError

A = np.array([
    [2, -1, 1],
    [-1, 10**-10, 10**-10],
    [1, 10**-10, 10**-10]
])

b = np.array([2*(1+10**-10), -10**-10, 10**-10])


# Attempt standard inverse
try:
    computed_A_inv = inv(A)
    print("\ncomputed inverse A:\n", computed_A_inv)
except LinAlgError:
    print("\nMatrix is singular or near-singular, using pseudoinverse instead.")


# mathematical solution
A_inv = np.array([
    [0, -1, 1],
    [-1, 0.5*10**10-1, 0.5*10**10+1],
    [1, 0.5*10**10+1, 0.5*10**10-1]
]) * 0.5


# condition number
K = np.linalg.norm(A, ord=np.inf) * np.linalg.norm(A_inv, ord=np.inf) 

print("\nA_inv:")
print(A_inv)
print("\nA:")
print(A)
print("\nA@A_inv:")
print(A@A_inv) # more precise
print("\nA@ computed(A_inv):")
print(A@computed_A_inv) # numberically unstable
print("\ncondition number k(A): ", K)




computed inverse A:
 [[ 0.00000000e+00 -5.00000000e-01  5.00000000e-01]
 [-5.00000000e-01  2.49999979e+09  2.49999979e+09]
 [ 5.00000000e-01  2.49999979e+09  2.49999979e+09]]

A_inv:
[[ 0.0e+00 -5.0e-01  5.0e-01]
 [-5.0e-01  2.5e+09  2.5e+09]
 [ 5.0e-01  2.5e+09  2.5e+09]]

A:
[[ 2.e+00 -1.e+00  1.e+00]
 [-1.e+00  1.e-10  1.e-10]
 [ 1.e+00  1.e-10  1.e-10]]

A@A_inv:
[[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.32450679e-17]
 [0.00000000e+00 4.97103078e-18 1.00000000e+00]]

A@ computed(A_inv):
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000035e-20  9.99999959e-01 -4.14201820e-08]
 [ 1.00000035e-20 -4.14201820e-08  9.99999959e-01]]

condition number k(A):  20000000002.0


## Perturbed Problem

$$
Ax = b
$$

$$
(A + E)y = b
$$

The relative error is given by:

$$
\frac{\|x - y\|}{\|x\|}
$$

Assume component-wise bound $|E| < \epsilon |A|$, $\epsilon = 10^{-8}$


In [2]:
import numpy as np

n = len(A)
eps = 10**-8
eps_matrix = (np.random.random((n,n))*2.0-1.0) * eps 

E = A * eps_matrix
A_perturbed = A+E

x = np.linalg.solve(A, b)
y = np.linalg.solve(A_perturbed, b)
relative_error = np.linalg.norm(x-y, ord=np.inf)/np.linalg.norm(x, ord=np.inf)
print("\nA")
print(A)
print("\nE")
print(E)
print("\nabs(E) < 10**-8 * abs(A) ?:")
print(abs(E) < 10**-8 * abs(A) )
print("\nComputed solution x to Ax=b:")
print(x)
print("\nComputed solution y to (A+E)y=b")
print(y)
print("\nRelative error: ", relative_error) # error is small despite ill-conditioned matrix A



A
[[ 2.e+00 -1.e+00  1.e+00]
 [-1.e+00  1.e-10  1.e-10]
 [ 1.e+00  1.e-10  1.e-10]]

E
[[ 6.13706559e-09 -4.87035014e-09  3.11892262e-10]
 [-4.85236387e-09  3.91463068e-19  8.61834307e-19]
 [-5.98453389e-09 -6.51768999e-19 -2.62546078e-19]]

abs(E) < 10**-8 * abs(A) ?:
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]

Computed solution x to Ax=b:
[ 1.00000008e-10 -1.00000000e+00  1.00000000e+00]

Computed solution y to (A+E)y=b
[ 9.99999802e-11 -1.00000001e+00  9.99999983e-01]

Relative error:  1.669386761992836e-08


## Scaled System

Let's scale the ill-conditioned matrix $A$ into a well-conditioned one using a diagonal matrix $D$.

$$
D = \text{diag}(10^{-5}, 10^5, 10^5)
$$

The new matrix $\tilde{A}$ is defined as:

$$
\tilde{A} := D A D
$$

$k(\tilde{A}) $ is not large, so $\tilde{A}$ is well-conditioned.

## scaled linear equation

- solve $(DAD)\mathbf{y} = Db \quad (\iff AD\mathbf{y} = b)$
- compute $\mathbf{x} = D\mathbf{y}$



In [3]:
import numpy as np

D = np.diag([10**-5, 10**5, 10**5])
D_inv = np.linalg.inv(D)
DAD_condition = np.linalg.norm(D@A@D, ord=np.inf) * np.linalg.norm(D_inv@A_inv@D_inv, ord=np.inf)

# solve  Ax=b
b = np.array([2*(1+10**-10), -10**-10, 10**-10])
A_tilder = D@A@D
y = np.linalg.solve(A_tilder, D@b)
x_scaled_sys = D@y
x_computed_by_np = np.linalg.solve(A, b)

# compute relative errors
true_x = np.array([10**-10, -1, 1]) # math answer
error_np = np.linalg.norm((true_x - x_computed_by_np), ord=np.inf)/ np.linalg.norm(true_x, ord=np.inf)
error_scaled_sys = np.linalg.norm(true_x - x_scaled_sys, ord=np.inf)/np.linalg.norm(true_x, ord=np.inf)

print("\nD")
print(D)
print("\nD_inv")
print(D_inv)
print("\nDAD:")
print(D@A@D)
print("\nCondition number of DAD:")
print(DAD_condition)
print("\ny:")
print(y)
print("\nSolution x to DADx=b:")
print(x_scaled_sys)
print("\nRelative error of x computed by numpy")
print(error_np) # larger
print("\nRelative error of x computed by the scaled system")
print(error_scaled_sys) # smaller



D
[[1.e-05 0.e+00 0.e+00]
 [0.e+00 1.e+05 0.e+00]
 [0.e+00 0.e+00 1.e+05]]

D_inv
[[1.e+05 0.e+00 0.e+00]
 [0.e+00 1.e-05 0.e+00]
 [0.e+00 0.e+00 1.e-05]]

DAD:
[[ 2.e-10 -1.e+00  1.e+00]
 [-1.e+00  1.e+00  1.e+00]
 [ 1.e+00  1.e+00  1.e+00]]

Condition number of DAD:
3.0

y:
[ 1.e-05 -1.e-05  1.e-05]

Solution x to DADx=b:
[ 1.e-10 -1.e+00  1.e+00]

Relative error of x computed by numpy
1.000000082740371e-10

Relative error of x computed by the scaled system
1.2924697071141057e-26


## Reference

* matrix computation 4th edition
* https://en.wikipedia.org/wiki/Condition_number
* https://en.wikipedia.org/wiki/Round-off_error
* https://en.wikipedia.org/wiki/Numerical_stability
